In [2]:
import os
import numpy as np
import csv
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image, ImageDraw
import glob 
import torch

In [47]:
train_path = '/home/tingwei/signboard/test/train' ## The path of train data
darknet_path = '/home/tingwei/signboard/darknet' ## The root path of YoloV4(darkenet)

test_path = '/home/tingwei/signboard/private/img_private' ## The path of test data
test_csv_path = '/home/tingwei/signboard/private/Task2_Private_String_Coordinate.csv' ## The path of test csv

model_path = '/home/tingwei/signboard/team_sixyeartaipeiman/best.pth' # The path of trained model weight

In [48]:
merge_path = os.path.join(train_path, 'merge_training')

In [49]:
train_df = pd.read_csv(os.path.join(merge_path, 'merge_training_all_handwrite.csv'))
val_df = pd.read_csv(os.path.join(merge_path, 'merge_validation_all_handwrite.csv'))

all_data_list = list(train_df['0']) + list(val_df['0'])
all_label_list = list(train_df['1']) + list(val_df['1'])

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
all_label_list_encoded = labelencoder.fit_transform(all_label_list)

label_dict = {}
for i in range(len(all_label_list_encoded)):
    label_dict[all_label_list[i]] = all_label_list_encoded[i]

In [50]:
device = torch.device('cuda')

from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=max(all_label_list_encoded) + 1)

model.load_state_dict(torch.load(model_path, map_location=device))
model = model.cuda()

Loaded pretrained weights for efficientnet-b7


In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo

from sklearn.utils import shuffle

from tqdm.notebook import tqdm as tqdm
import time
import matplotlib.pyplot as plt
import time
import os
import copy
from natsort import natsorted as natsort
import random
import math
import cv2
from torch.backends import cudnn
# cudnn.benchmark = True # fast training
from torch.cuda.amp import autocast as autocast

plt.ion()   # interactive mode

from scipy.io import loadmat
from PIL import Image
from torch.utils.data import Dataset, DataLoader

class DatasetLoader(Dataset):
    def __init__(self, input_, transform, tag, label_dict):
        self.transform = transform
        self.tag = tag
        self.input_ = input_


        self.classes = list(label_dict.keys())
        self.class_to_idx = label_dict        

        images_names = self.input_
        self.imgs = [img for i, img in enumerate(images_names)]

    def __getitem__(self, index):
        img = self.imgs[index]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
        if self.transform is not None:
            img = Image.fromarray(img).convert('RGB')
            img = self.transform(img)
        return img

        
    def __len__(self):
        return len(self.imgs)


In [17]:
import json
from tqdm.notebook import tqdm

all_label = []
all_points = []
all_names = []
all_width = []
all_height = []
# 開啟 JSON 檔案
with open(os.path.join(darknet_path, 'result.json')) as f:
    # 讀取 JSON 檔案
    p = json.load(f)

In [27]:
from PIL import Image, ImageDraw
import numpy as np

df_test = pd.read_csv(test_csv_path, header = None)
test_name_list = list(df_test[0])
save_test_path = os.path.split(test_path)[0]


for i in tqdm(range(len(p))):
    name = p[i]['filename']
    public_save_index = 0
    for public_i in range(len(test_name_list)):
        if test_name_list[public_i] == os.path.split(name)[-1].split('.')[0]:
            row = list(df_test.iloc[public_i])
            
            predict_list = []
            tmp_img = cv2.imread(os.path.join(test_path, str(row[0]) + '.jpg'))
            h, w, _ = tmp_img.shape

            mask = Image.new('L', [w, h], 0)

            crop_position = []
            tmp_list = []
            for k, position in enumerate(row):
                if k == 0:
                    continue
                elif k % 2 == 1:
                    if int(position) > w:
                        position = w
                    elif int(position) <= 0:
                        position = 1
                elif k % 2 == 0:
                    if int(position) > h:
                        position = w
                    elif int(position) <= 0:
                        position = 1
                    
                tmp_list.append(int(position))
                if k % 2 == 0:
                    crop_position.append(tuple(tmp_list))
                    tmp_list = []

            draw = ImageDraw.Draw(mask, 'L')
            draw.polygon(crop_position, fill = 255)
            
            x_list = [x[0] for x in crop_position]
            y_list = [x[1] for x in crop_position]
            x_range = max(x_list) - min(x_list)
            y_range = max(y_list) - min(y_list)
            
            mask = np.array(mask)
            
            if not os.path.exists(os.path.join(save_test_path, 'Yolo_predicted'
                                                       , os.path.split(name)[-1].split('.')[0] + '_' + str(str(public_save_index)))):
                        os.makedirs(os.path.join(save_test_path, 'Yolo_predicted', os.path.split(name)[-1].split('.')[0] 
                                                 + '_' + str(str(public_save_index))))

            tmp_dict = {}
            save_list = []
            save_list.append(name)
            tmp_j = 0
            for j in range(len(p[i]['objects'])):
                c_x = p[i]['objects'][j]['relative_coordinates']['center_x']
                c_y = p[i]['objects'][j]['relative_coordinates']['center_y']
                yolo_w = p[i]['objects'][j]['relative_coordinates']['width']
                yolo_h = p[i]['objects'][j]['relative_coordinates']['height']

                xmin = round(((2 * w * c_x) - (w * yolo_w)) / 2)
                xmax = round(((2 * w * c_x) + (w * yolo_w)) / 2)
                ymin = round(((2 * h * c_y) - (h * yolo_h)) / 2)
                ymax = round(((2 * h * c_y) + (h * yolo_h)) / 2)
                
                bbx_mask = np.zeros([mask.shape[0], mask.shape[1]], dtype='uint8')
                bbx_mask[ymin:ymax, xmin:xmax] = 255

                if np.sum((mask//255) * (bbx_mask//255)) / np.sum(bbx_mask//255) >= 0.5:
                    tmp_dict[tmp_j] = [xmin, ymin, xmax, ymax]
                    tmp_j += 1
                
            if x_range >= y_range:
                tmp_dict = sorted(tmp_dict.items(), key=lambda x:x[1][0]) # sort by the bounding box of xmin
            else:
                tmp_dict = sorted(tmp_dict.items(), key=lambda x:x[1][1]) # sort by the bounding box of ymin

            for dic in range(len(tmp_dict)):
                for dic_ele in tmp_dict[dic][1]:
                    save_list.append(str(dic_ele))
                    
            tmp_list = []
                    
            if len(save_list) != 1:
                for j in range(len(save_list)):
                    if j == 0:
                        name = save_list[j]
                        pure_name = os.path.split(name)[-1]
                    elif j % 4 != 0:
                        if int(save_list[j]) < 0:
                            save_list[j] = 0
                        tmp_list.append(int(save_list[j]))
                    else:
                        if int(save_list[j]) < 0:
                            save_list[j] = 0
                        tmp_list.append(int(save_list[j]))
                        save_img = tmp_img[tmp_list[1]:tmp_list[3], tmp_list[0]:tmp_list[2]]
                        tmp_list = []
                        cv2.imwrite(os.path.join(save_test_path, 'Yolo_predicted', str(os.path.split(name)[-1].split('.')[0]) + '_' + str(public_save_index), str(j//4) + '.png'), save_img)
            public_save_index += 1

/home/tingwei/PycharmProjects/General/venv/lib/python3.6/site-packages/ipykernel_launcher.py:76: RuntimeWarning: invalid value encountered in ulong_scalars


In [28]:
yolo_path = os.path.join(save_test_path, 'Yolo_predicted')
yolo_list = natsort(os.listdir(yolo_path))

In [29]:
all_output_list = []
img_list = []
img_count_list = []
for img_folder in tqdm(yolo_list):
    folder_list = natsort(os.listdir(os.path.join(yolo_path, img_folder)))
    if len(folder_list) == 0:
        output_chinese = '###'
        all_output_list.append(output_chinese)
    else:
        for predict_i, img_name in enumerate(folder_list):
            img = cv2.imread(os.path.join(yolo_path, img_folder, img_name))
            img_list.append(img)
    img_count_list.append(len(folder_list))

In [52]:
train_on_gpu = True
data_transforms = {
                  'test':transforms.Compose([
#                       transforms.Resize((128, 128), interpolation=torchvision.transforms.InterpolationMode.NEAREST),
                      transforms.Resize((256, 256)),
#                       transforms.RandomCrop(256),
#                       transforms.CenterCrop((256, 256)),
#                       transforms.Grayscale(num_output_channels=1),
                      transforms.ToTensor(),
#                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                      ])
                  }

image_datasets = {'test':DatasetLoader(img_list, data_transforms['test'], 'test', label_dict)
                 }

test_batch_size = 32
All_loader = {'test':DataLoader(dataset=image_datasets['test'], batch_size=test_batch_size, shuffle=False, num_workers=4, pin_memory=True),
             }

In [53]:
all_output_list = []
train_on_gpu = True
if train_on_gpu:
    model = model.cuda()

model.eval()
with torch.no_grad():
    for data in tqdm(All_loader['test']):
        if train_on_gpu:
            data = data.cuda()

        output = model(data)

        _, predicted = torch.max(output.data, 1)
        all_output_list += list(predicted.cpu().detach().numpy())

In [54]:
def get_key (dict_, value):
    return [k for k, v in dict_.items() if v == value]

In [55]:
all_chinese_output = []
now_index = 0
for i in tqdm(img_count_list):
    if i == 0:
        output_chinese = '###'
    else:
        tmp_output = (all_output_list[now_index:now_index + i])
        for j, label in enumerate(tmp_output):
            if j == 0:
                output_chinese = get_key(label_dict, label)[0]
            else:
                output_chinese += get_key(label_dict, label)[0]
        now_index += i
#     print(output_chinese)
    all_chinese_output.append(output_chinese) 

In [19]:
output_df = pd.read_csv(test_csv_path, header = None)
all_output_df = pd.concat([output_df, pd.Series(all_chinese_output).rename(9)], axis=1) # 增加列t

In [21]:
all_output_df.to_csv(os.path.join(os.path.split(model_path)[0], 'output.csv'), header = None, index = False)